# Implement Chroma DB
This might be an efficient way to stash documents and select them for the AI to analyze.

* https://docs.trychroma.com/getting-started

In [ ]:
%pip install chromadb
%pip freeze > requirements.txt

In [2]:
import chromadb
import json

chroma_client = chromadb.PersistentClient(path="llama-data/embeddings/chroma")

In [5]:
raw = open("./llama-data/data/apache_logs/3.txt").read().strip().split("\n")

#  https://www.geeksforgeeks.org/break-list-chunks-size-n-python/
chunk = 10
all_documents = [
    ((x * chunk) // chunk, raw[x * chunk : (x + 1) * chunk])
    for x in range((len(raw) + chunk - 1) // chunk)
]

for index, documents in all_documents:
    chroma_client.delete_collection(name=f"vec-{index}")
    collection = chroma_client.get_or_create_collection(name=f"vec-{index}")

    ids = range(len(documents), len(documents) * 2)
    collection.add(documents=documents, ids=[f"id-{x+1}" for x in ids])

In [6]:
results = []
anomaly_count = 2

for index, _ in all_documents:
    collection = chroma_client.get_collection(name=f"vec-{index}")
    document_count = len(collection.get()["ids"])
    iterations = round(document_count / anomaly_count)
    filter_out = []
    query = "*"

    for i in range(iterations):
        print(f"Index: {index+1}/{len(all_documents)}, Iteration: {i+1}/{iterations}")

        if len(filter_out) == 0:
            result = collection.query(
                query_texts=[query],
                n_results=anomaly_count,
            )
        
        else:
            exclusions = {"$and": [{"$not_contains": x} for x in filter_out]}
            result = collection.query(
                query_texts=[query],
                n_results=anomaly_count,
                where_document=exclusions,
            )
            
        query = result["documents"][0][0]
        [print(f"Filtering: {x[:15]}...") for x in result["documents"][0]]
        [filter_out.append(x) for x in result["documents"][0]]
        
        # This will be overwritten each round except at the end.
        results = result["documents"][0]


print(json.dumps(results, indent=4))

Index: 1/10, Iteration: 1/5
Filtering: 146.1.1.2 - - [...
Filtering: 146.1.1.2 - - [...
Index: 1/10, Iteration: 2/5
Filtering: 146.1.1.2 - - [...
Filtering: 146.1.1.2 - - [...
Index: 1/10, Iteration: 3/5
Filtering: 146.1.1.2 - - [...
Filtering: 146.1.1.2 - - [...
Index: 1/10, Iteration: 4/5
Filtering: 212.201.44.247 ...
Filtering: 212.201.44.247 ...
Index: 1/10, Iteration: 5/5
Filtering: 178.255.215.83 ...
Filtering: 66.249.73.135 -...
Index: 2/10, Iteration: 1/5
Filtering: 85.170.84.143 -...
Filtering: 85.170.84.143 -...
Index: 2/10, Iteration: 2/5
Filtering: 85.170.84.143 -...
Filtering: 85.170.84.143 -...
Index: 2/10, Iteration: 3/5
Filtering: 85.170.84.143 -...
Filtering: 66.249.73.135 -...
Index: 2/10, Iteration: 4/5
Filtering: 82.193.99.33 - ...
Filtering: 82.193.99.33 - ...
Index: 2/10, Iteration: 5/5
Filtering: 82.193.99.33 - ...
Filtering: 50.16.19.13 - -...
Index: 3/10, Iteration: 1/5
Filtering: 46.105.14.53 - ...
Filtering: 50.131.51.216 -...
Index: 3/10, Iteration: 2/5
Filt